# Day 6 - Feature Engineering
### Machine Learning Roadmap - Week 2 
### Author - N Manish Kumar 
---

Feature engineering is the process of transforming raw data into features that better represent the underlying problem.

In this notebook, we apply feature engineering techniques to improve a Logistic Regression model without changing the algorithm.

## 1. Load Dataset

We use the Breast Cancer dataset from scikit-learn.

This step:
- Loads the raw dataset
- Converts it into a pandas DataFrame
- Explicitly separates features (X) and target (y)

All feature engineering will be applied only to X.

In [6]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

# Load dataset
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

# Train-test split (consistent with previous days)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

### Scale Features

In [7]:
scaler = StandardScaler()

X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

### Training and Evaluating Baseline Model

In [8]:
# Train Baseline Model
baseline_model = LogisticRegression(C=1.0, max_iter=500)
baseline_model.fit(X_train_s, y_train)

# Evaluate Baseline Model
y_pred_base = baseline_model.predict(X_test_s)

baseline_acc = accuracy_score(y_test, y_pred_base)
baseline_f1 = f1_score(y_test, y_pred_base)

baseline_acc, baseline_f1

(0.9824561403508771, 0.9861111111111112)

### Interpretation
The baseline model provides a reference point.

Any improvement after feature engineering should be measured relative to this baseline, not in isolation.

---
## 2. Feature Engineering #1 : Interaction Features
Sometimes the target depends not on a single feature, but on how two features interact with each other.

Interaction features allow a linear model to capture relationships of the form: feature_A × feature_B

### Create Interaction Feature

In [9]:
# Create feature-engineered copies
X_train_fe = X_train.copy()
X_test_fe = X_test.copy()

# Interaction feature: mean radius × mean texture
X_train_fe["radius_texture_interaction"] = (
    X_train_fe["mean radius"] * X_train_fe["mean texture"]
)

X_test_fe["radius_texture_interaction"] = (
    X_test_fe["mean radius"] * X_test_fe["mean texture"]
)

### Scale Engineered Features

In [10]:
# Scale Engineered Features
scaler_fe_1 = StandardScaler()
X_train_fe_scaled = scaler_fe_1.fit_transform(X_train_fe)
X_test_fe_scaled = scaler_fe_1.transform(X_test_fe)

### Train Logistic Regression on Engineered Features

In [11]:
# Train logistic regression on engineered features
interaction_model = LogisticRegression(C=1.0, max_iter=500)
interaction_model.fit(X_train_fe_scaled, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

### Evalueate Model

In [13]:
# Predictions
y_pred_inter = interaction_model.predict(X_test_fe_scaled)

# Metrics
interaction_acc = accuracy_score(y_test, y_pred_inter)
interaction_f1 = f1_score(y_test, y_pred_inter)

interaction_acc, interaction_f1

(0.9824561403508771, 0.9861111111111112)

### Interpretation

Compared to the baseline:
- Accuracy: unchanged
- F1 Score: unchanged

The interaction feature helped/did not help because:
    After adding the interaction feature (mean radius × mean texture),
the accuracy and F1 score remained unchanged.

This suggests that the interaction did not introduce
new discriminative information beyond what the original
features already provided.

The baseline Logistic Regression model already performs strongly
on this dataset, and regularization likely suppressed the
interaction feature due to its limited additional contribution.

---

## 3. Feature Engineering #2 : Log Transform
Some features have highly skewed distributions.

Log transformation compresses large values and
expands small values, making patterns easier
for linear models to learn.

### Create Log Feature

In [14]:
# Add log-transformed feature
X_train_fe_log = X_train_fe.copy()
X_test_fe_log = X_test_fe.copy()

# Log-transform a skewed feature
X_train_fe_log["log_mean_area"] = np.log1p(X_train_fe_log["mean area"])
X_test_fe_log["log_mean_area"] = np.log1p(X_test_fe_log["mean area"])

### Scale Log-Engineered Features

In [15]:
# Scale features
scaler_fe_2 = StandardScaler()
X_train_fe_log_scaled = scaler_fe_2.fit_transform(X_train_fe_log)
X_test_fe_log_scaled = scaler_fe_2.transform(X_test_fe_log)

### Train Logistic Regression on Engineered Features

In [17]:
# Train logistic regression with log-transformed features
log_model = LogisticRegression(C=1.0, max_iter=500)
log_model.fit(X_train_fe_log_scaled, y_train)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

### Evaluating Model

In [19]:
# Predictions
y_pred_log = log_model.predict(X_test_fe_log_scaled)

# Metrics
log_acc = accuracy_score(y_test, y_pred_log)
log_f1 = f1_score(y_test, y_pred_log)

log_acc, log_f1

(0.9824561403508771, 0.9861111111111112)

### Log Transformation Results

After applying a log transformation to `mean area`,
the model’s accuracy and F1 score remained unchanged.

This indicates that the original feature already captured
most of the useful information, and the log-transformed
version did not introduce additional discriminative power.

Since the Breast Cancer dataset is relatively well-separated
and features are well-scaled, log transformation provided
minimal benefit in this case.

---

## 4. Error Analysis (Andrew Ng Core Skill)

When feature engineering does not improve performance,
the correct next step is to analyze model errors.

Instead of adding random features, we inspect
misclassified examples to understand:
- Where the model fails
- What information might be missing

### Get Predictions + Errors

In [21]:
# Baseline predictions (already trained in Step 1)
y_pred_base = baseline_model.predict(X_test_s)

# Identify misclassified indices
misclassified_idx = np.where(y_pred_base != y_test.values)[0]

len(misclassified_idx)

2

### Inspect Misclassified Samples

In [22]:
# Inspect misclassified examples
misclassified_samples = X_test.iloc[misclassified_idx]
misclassified_samples.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
541,14.47,24.99,95.81,656.4,0.08837,0.123,0.10090,0.03890,0.1872,0.06341,...,16.22,31.73,113.5,808.9,0.1340,0.4202,0.4040,0.1205,0.3187,0.1023
73,13.80,15.79,90.43,584.1,0.10070,0.128,0.07789,0.05069,0.1662,0.06566,...,16.57,20.86,110.3,812.4,0.1411,0.3542,0.2779,0.1383,0.2589,0.1030


### Compare With Correct Predictions

In [24]:
# Correctly classified samples
correct_idx = np.where(y_pred_base == y_test.values)[0]
correct_samples = X_test.iloc[correct_idx]

correct_samples.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
count,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,...,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000
mean,14.371223,19.441786,93.620714,681.296429,0.097176,0.105881,0.087188,0.051330,0.183451,0.062677,...,16.640545,25.787768,109.761786,928.780357,0.132437,0.251633,0.253127,0.117193,0.286890,0.083617
std,3.658738,3.853911,25.148977,382.284527,0.016427,0.050446,0.071990,0.038724,0.024763,0.006734,...,5.128370,5.850991,35.532756,636.684533,0.024230,0.147567,0.177201,0.062076,0.049757,0.016641
min,7.760000,11.970000,47.920000,181.000000,0.052630,0.031160,0.000000,0.000000,0.135300,0.050440,...,9.456000,14.100000,59.160000,268.600000,0.088640,0.052320,0.000000,0.000000,0.199900,0.059330
25%,11.867500,16.695000,76.327500,433.325000,0.085218,0.066275,0.037465,0.021698,0.163950,0.057985,...,13.017500,21.317500,84.947500,515.350000,0.119750,0.148375,0.121175,0.073483,0.252325,0.071080
50%,13.650000,19.025000,87.600000,571.050000,0.096870,0.096715,0.065190,0.041650,0.184350,0.062225,...,14.970000,25.430000,97.965000,688.250000,0.131150,0.214300,0.202950,0.108750,0.282250,0.080245
75%,16.327500,21.887500,106.675000,819.800000,0.107525,0.131675,0.118525,0.074845,0.197600,0.065795,...,19.632500,30.372500,128.825000,1169.250000,0.145700,0.334100,0.375600,0.157550,0.307425,0.090740
max,27.420000,29.330000,186.900000,2501.000000,0.163400,0.277000,0.363500,0.187800,0.259500,0.095020,...,36.040000,41.850000,251.200000,4254.000000,0.222600,0.868100,0.938700,0.268800,0.475300,0.143100


### Error Analysis Observations

After inspecting misclassified samples, I observed that:
- Certain features tend to have overlapping values between classes
- Some samples lie near the decision boundary
- The model struggles with borderline or ambiguous cases

This suggests that:
- Linear separation may be insufficient for some regions
- More informative features or non-linear models may be required

----
# Feature Engineering Takeaway

Feature engineering did not significantly improve performance
because the baseline model already performs strongly on this dataset.

Error analysis indicates that remaining errors are due to
inherent overlap between classes rather than missing features.

This suggests diminishing returns from further feature engineering
with Logistic Regression on this dataset.